# Infrared Data Analysis

The workflow we have here does the following:

 - First, we read the data and perform some basic 'John Snow' decomposition, like PCA, KernelPCA or ICA.
 
 -The results of this decomposition is used to construct an initial set of component spectra that are used to kickstart and MCR-ALS decomposition.
 - The results of this decomposition is displayed in the IRViz application


In [1]:
%pip install pymcr  # install analysis package not included in IRViz dependencies

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'
You should consider upgrading via the 'c:\users\lbl\.virtualenvs\ryujin\scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from dask import array as da
import h5py as h5
import numpy as np
import sklearn.decomposition
from irviz.app import open_map_file, open_ir_file
import phonetic_alphabet as pa

from pymcr.mcr import McrAR
from pymcr.regressors import OLS, NNLS
from pymcr.constraints import ConstraintNonneg, ConstraintNorm
import logging, sys


### Read in the data

In [2]:
# Open the data
data_file =  "E:/BP-area3a.h5"
data, bounds = open_map_file(data_file)

### Do the initial decomposition

In [3]:
n_components = 5
method = "KernelPCA"

models = {"KernelPCA": sklearn.decomposition.KernelPCA(kernel="rbf", 
                                        fit_inverse_transform=True, 
                                        n_components=n_components),
        "PCA": sklearn.decomposition.PCA(n_components=n_components),
        "ICA": sklearn.decomposition.FastICA(n_components=n_components) }


model = models[method]

decomposition = model.fit_transform(
    data.transpose(1,2,0).reshape(-1, data.shape[0])).T.reshape(-1, *data.shape[1:])
cluster_labels = np.argmax(decomposition, axis=0)

cluster_label_names = []
label_letters = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
for label_letter in label_letters[:n_components]:
    cluster_label_names.append( pa.read(label_letter) )

decomposition_components = None
try:
    decomposition_components = model.components_
except:
    pass

mean_spectra = []
for ii in range(n_components):
    sel = cluster_labels.flatten() == ii
    sel = np.where(sel)[0]    
    spectra = data.reshape( (data.shape[0],data.shape[1]*data.shape[2]) )[:,sel]
    spectra = np.mean( spectra, axis = 1)
    mean_spectra.append( spectra ) 

mean_spectra = np.vstack(mean_spectra)
nan_list = np.isnan(mean_spectra.compute())[:,0]
n_nans = len(np.where(nan_list)[0])
if n_nans >0 :
    print("Please reduce the number of components to ",n_components - n_nans)


### Here we use the pyMCR package to do MCR analyses

We kickstart this by the decomposition done above. This calculation can take a little bit of time.


In [4]:
from io import StringIO
mystdout = StringIO()
logger = logging.getLogger('pymcr')
logger.setLevel(logging.INFO)
stdout_handler = logging.StreamHandler(stream=mystdout)

# Set the message format. Simple and removing log level or date info
stdout_format = logging.Formatter('%(message)s')  # Just a basic message akin to print statements
stdout_handler.setFormatter(stdout_format)
logger.addHandler(stdout_handler)

mcrar = McrAR(max_iter=50, st_regr='NNLS', c_regr=NNLS(), 
              c_constraints=[ConstraintNonneg()]) 

# because we use dask arrays, and pymcr wants numpy, some jiggery pokery is happening.
mcrar.fit(data.transpose(1,2,0).reshape(-1, data.shape[0]).compute().astype(float), 
          ST=mean_spectra.compute().astype(float), verbose=False)

# now we are done
concentrations = mcrar.C_opt_.T.reshape( (n_components, data.shape[1], data.shape[2]) )
components = mcrar.ST_opt_
# get cluster labels based on the maximum component available
cluster_labels = np.argmax(concentrations, axis=0)

### Renormalize

It makes life easier if we normalize each spectra such that its integrated intensity is equal to mean spectrum. We subsequently rescale the coefficients as well


In [5]:
# first normalize the spectra
mean_norm = np.mean( np.sum( data, axis = 0) )
norms = np.sum( components, axis =1 )/mean_norm
components = (components.T / norms).T

# now rescale the concentrations
for ii in range(n_components):
    concentrations[ii,:]=concentrations[ii,:]/norms[ii]

## Create a Viewer
Vizualize the results in the IRViz app


In [6]:
from irviz.viewer import Viewer
viewer = Viewer(data=data,
                         decomposition=concentrations,
                         bounds=bounds,
                         x_axis_title='X (μm)',
                         y_axis_title='Y (μm)',
                         spectra_axis_title='Wavenumber (cm⁻¹)',
                         intensity_axis_title='Intensity',
                         invert_spectra_axis=True,
                         cluster_labels=cluster_labels,
                         cluster_label_names=cluster_label_names,
                         component_spectra=components).run_embedded(run_kwargs=dict(height=1500, mode="inline"))

## Isolate Background
Use the IRViz app to interactively determine parameters for background removal

First, lets define a function to describe our background

In [8]:
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, ConstantKernel as C


def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx


def GPR_based_background_single_spectrum(wavenumbers,
                                         spectrum,
                                         control_points,
                                         control_regions,
                                         mask, # ????????
                                         rbf_start=1000,
                                         rbf_low=500,
                                         rbf_high=1e8,
                                         C_start=1.0,
                                         C_low=1e-6,
                                         C_high=1e4
                                         ):
    """
    Build a background model using GPR

    :param wavenumbers: Input wavenumbers
    :param spectrum: input spectrum
    :param control_points: input control points, poicked manually
    :param rbf_kernel_params: kernel parameters, defaults ok
    :param constant_kernel_params: kernel parameters, defaults ok
    :return: a fitted background.
    """
    # gather the x values
    these_idxs = []
    for cp in control_points:
        these_idxs.append( find_nearest(wavenumbers, cp) )
    these_idxs = np.array(these_idxs)
    these_x = wavenumbers[these_idxs]
    these_y = spectrum[these_idxs]
    kernel = C(C_start,
               (C_low,
                C_high)) * \
             RBF(rbf_start, (rbf_low, rbf_high))

    gpr = gaussian_process.GaussianProcessRegressor(kernel=kernel).fit(these_x.reshape(-1,1),
                                                                       these_y.reshape(-1,1))
    tmp_bg = gpr.predict(wavenumbers.reshape(-1,1))
    return tmp_bg.flatten()

In [9]:
from irviz import BackgroundIsolator
background_isolator = BackgroundIsolator(data=data, bounds=bounds, background_function=GPR_based_background_single_spectrum).run_embedded(run_kwargs=dict(height=1000))

In [10]:
background_isolator.parameter_sets

[]